In [ ]:
# https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects

In [ ]:
# Autoreloads external files without having to restart the notebook
%load_ext autoreload
%autoreload 2

In [ ]:
!ls ../input

In [ ]:
!ls ../input/darknet-wheat/
!ls ../input/darknet-gpu/

In [ ]:
!cp -r ../input/darknet-wheat/* .

!cp ../input/darknet-gpu/darknet .
!cp ../input/darknet-gpu/darknet.py .
!cp ../input/darknet-gpu/libdarknet.so .

!chmod a+x ./darknet
!ls -la .

In [ ]:
# !cat darknet.py
# !./darknet

In [ ]:
import sys
sys.path.insert(0, "../input/weightedboxesfusion/")

import pandas as pd
import numpy as np
import os
from datetime import datetime
import time
import random
import json
import collections
import shutil as sh

from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

from sklearn.model_selection import StratifiedKFold

import darknet as dn
from ensemble_boxes import *

# from skopt import gp_minimize, forest_minimize
# from skopt.utils import use_named_args
# from skopt.plots import plot_objective, plot_evaluations, plot_convergence, plot_regret
# from skopt.space import Categorical, Integer, Real

import torch

import gc
gc.enable()

np.set_printoptions(suppress=True)

print('PyTorch version', torch.__version__)

SEED = 1120
# SEED = 42

image_width = 1024
image_height = 1024

batch=64
# subdivisions=16
subdivisions=32
# subdivisions=64
burn_in = 0

# Prediction confidence score
prob_threhold = 0.5

# WBF
# iou_thr = 0.55
# skip_box_thr = 0.43
iou_thr = 0.55
skip_box_thr = 0.1

model_image_sizes = [1024, 1024, 1024, 1024, 1024]

# [Local Env]
# dataset_folder = "/workspace/Kaggle/Wheat"
# train_image_folder = "/workspace/Kaggle/Wheat/train"
# test_image_folder = "/workspace/Kaggle/Wheat/test"

# meta_file = b"/workspace/Github/darknet/build/darknet/x64/data/wheat_notebook.data"
# model_cfg_files = [
#     b"./darknet_wheat/yolov4-v3-20000iter/yolov4-mish-416-wheat-v3.cfg",
#     b"./darknet_wheat/yolov4-v3-20000iter-fold1/yolov4-mish-416-wheat-v3-fold1.cfg",
#     b"./darknet_wheat/yolov4-v3-20000iter-fold2/yolov4-mish-416-wheat-v3-fold2.cfg",
#     b"./darknet_wheat/yolov4-v3-20000iter-fold3/yolov4-mish-416-wheat-v3-fold3.cfg",
#     b"./darknet_wheat/yolov4-v3-20000iter-fold4/yolov4-mish-416-wheat-v3-fold4.cfg",
# ]
# model_weights_files = [
#     b"./darknet_wheat/yolov4-v3-20000iter/yolov4-mish-416-wheat-v3_best.weights",
#     b"./darknet_wheat/yolov4-v3-20000iter-fold1/yolov4-mish-416-wheat-v3-fold1_best.weights",
#     b"./darknet_wheat/yolov4-v3-20000iter-fold2/yolov4-mish-416-wheat-v3-fold2_best.weights",
#     b"./darknet_wheat/yolov4-v3-20000iter-fold3/yolov4-mish-416-wheat-v3-fold3_best.weights",
#     b"./darknet_wheat/yolov4-v3-20000iter-fold4/yolov4-mish-416-wheat-v3-fold4_best.weights",
# ]

# [Kaggle Env]
dataset_folder = "../input/global-wheat-detection/"
train_image_folder = "../input/global-wheat-detection/train"
test_image_folder = "../input/global-wheat-detection/test"

meta_file = b"data/wheat.data"
model_cfg_files = [
#     b"./yolov4-v3-20000iter/yolov4-mish-416-wheat-v3.cfg",
#     b"./yolov4-v3-20000iter-fold1/yolov4-mish-416-wheat-v3-fold1.cfg",
    b"./yolov4-v3-20000iter-fold2/yolov4-mish-416-wheat-v3-fold2.cfg",
#     b"./yolov4-v3-20000iter-fold3/yolov4-mish-416-wheat-v3-fold3.cfg",
#     b"./yolov4-v3-20000iter-fold4/yolov4-mish-416-wheat-v3-fold4.cfg",
]
model_weights_files = [
#     b"./yolov4-v3-20000iter/yolov4-mish-416-wheat-v3_best.weights",
#     b"./yolov4-v3-20000iter-fold1/yolov4-mish-416-wheat-v3-fold1_best.weights",
    b"./yolov4-v3-20000iter-fold2/yolov4-mish-416-wheat-v3-fold2_best.weights",
#     b"./yolov4-v3-20000iter-fold3/yolov4-mish-416-wheat-v3-fold3_best.weights",
#     b"./yolov4-v3-20000iter-fold4/yolov4-mish-416-wheat-v3-fold4_best.weights",
]

# used for fast inference in submission
USE_OPTIMIZE = len(os.listdir(test_image_folder)) == 10  

# About 2310 iterations per epoch
if USE_OPTIMIZE:
    iteration_offset = 0
    pseudo_iterations = 10 # 4 minutes
    # pseudo_iterations = 50 # 20 minutes
else:
    iteration_offset = 0
    # pseudo_iterations = 600 # 4 hours
    pseudo_iterations = 1000


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(SEED)

In [ ]:
# https://www.kaggle.com/c/global-wheat-detection/discussion/159578
bad_images = [
    "41c0123cc",
    "a1321ca95",
    "2cc75e9f5",
    "42e6efaaa",
    "409a8490c",
    "d067ac2b1",
    "d60e832a5",
    "893938464",
]

bad_boxes = [
    3687, 117344, 173, 113947, 52868, 2159, 2169, 121633, 121634, 147504,
    118211, 52727, 147552
]

## Create Stratified K-Folds

In [ ]:
marking_df = pd.read_csv(f"{dataset_folder}/train.csv")

# replace nan values with zeros
marking_df['bbox'] = marking_df.bbox.fillna('[0,0,0,0]')

bboxs = np.stack(
    marking_df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    marking_df[column] = bboxs[:, i]
marking_df.drop(columns=['bbox'], inplace=True)

marking_df["id"] = marking_df.index.tolist()

marking_df['x_center'] = marking_df['x'] + marking_df['w'] / 2
marking_df['y_center'] = marking_df['y'] + marking_df['h'] / 2
marking_df['classes'] = 0

In [ ]:
# Drop bad bboxes
dataset_df = marking_df[~marking_df["id"].isin(bad_boxes)].copy()
assert dataset_df.shape[0] + len(bad_boxes) == marking_df.shape[0]
print(marking_df.shape, dataset_df.shape)
dataset_df = dataset_df[~dataset_df["image_id"].isin(bad_images)].copy()
print(marking_df.shape, dataset_df.shape)

In [ ]:
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=SEED)
skf

In [ ]:
df_folds = dataset_df[['image_id']].copy()
df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()

df_folds.loc[:, 'source'] = dataset_df[['image_id', 'source'
                                        ]].groupby('image_id').min()['source']

df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['source'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str))
df_folds.loc[:, 'fold'] = 0

for fold_number, (train_index, val_index) in enumerate(
        skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

df_folds = df_folds.reset_index()

## Utility Functions

In [ ]:
# https://www.kaggle.com/pestipeti/pytorch-starter-fasterrcnn-inference
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(
            j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)


def process_detection_result(bboxes, resize_width, prob_threhold=0.5):
    scores = np.zeros((len(bboxes)))
    numpy_bboxes = np.zeros((len(bboxes), 4))
    for i, bbox in enumerate(bboxes):
        scores[i] = bbox[1]
        cord = bbox[2]

        # Convert center (x,y, w, h) to (x1, y1, x2, y2)
        x1 = cord[0] - cord[2] / 2
        y1 = cord[1] - cord[3] / 2
        x2 = cord[0] + cord[2] / 2
        y2 = cord[1] + cord[3] / 2

        # Convert to original resolution
        x1 /= resize_width
        y1 /= resize_width
        x2 /= resize_width
        y2 /= resize_width

        x1 *= image_width
        y1 *= image_width
        x2 *= image_width
        y2 *= image_width

        x1 = int(round(max(min(x1, image_width - 1), 0)))
        y1 = int(round(max(min(y1, image_width - 1), 0)))
        x2 = int(round(max(min(x2, image_width - 1), 0)))
        y2 = int(round(max(min(y2, image_width - 1), 0)))

        numpy_bboxes[i, :] = np.array([x1, y1, x2, y2])

    indexes = np.where(scores > prob_threhold)
    numpy_bboxes = numpy_bboxes[indexes]
    scores = scores[indexes]

    return scores, numpy_bboxes

## TTA Functions

In [ ]:
class BaseWheatTTA:
    """ author: @shonenkov """
    image_size = image_width

    def augment(self, image):
        raise NotImplementedError

    def batch_augment(self, images):
        raise NotImplementedError

    def deaugment_boxes(self, boxes):
        raise NotImplementedError


class TTAHorizontalFlip(BaseWheatTTA):
    """ author: @shonenkov """
    def augment(self, image):
        image = np.fliplr(image)
        return image
        # return image.flip(1)

    def batch_augment(self, images):
        images = np.fliplr(images)
        return images
        # return images.flip(2)

    def deaugment_boxes(self, boxes):
        boxes[:, [0, 2]] = self.image_size - boxes[:, [2, 0]]
        return boxes


class TTAVerticalFlip(BaseWheatTTA):
    """ author: @shonenkov """
    def augment(self, image):
        image = np.flipud(image)
        return image
        # return image.flip(2)

    def batch_augment(self, images):
        images = np.flipud(images)
        return images
        # return images.flip(3)

    def deaugment_boxes(self, boxes):
        boxes[:, [3, 1]] = self.image_size - boxes[:, [1, 3]]
        return boxes


class TTARotate90(BaseWheatTTA):
    """ author: @shonenkov """
    def augment(self, image):
        image = np.rot90(image, k=1, axes=(0, 1))
        return image
        # return torch.rot90(image, 1, (1, 2))

    def batch_augment(self, images):
        images = np.rot90(images, k=1, axes=(1, 2))
        return images
        # return torch.rot90(images, 1, (2, 3))

    def deaugment_boxes(self, boxes):
        res_boxes = boxes.copy()
        res_boxes[:, [0, 2]] = self.image_size - boxes[:, [1, 3]]
        res_boxes[:, [1, 3]] = boxes[:, [2, 0]]
        return res_boxes


class TTARotate180(BaseWheatTTA):
    def augment(self, image):
        tmp = np.rot90(image, k=1, axes=(0, 1))
        tmp = np.rot90(tmp, k=1, axes=(0, 1))
        # tmp = np.rot90(image, k=1, axes=(1, 2))
        # tmp = np.rot90(tmp, k=1, axes=(1, 2))
        return tmp
        # tmp = torch.rot90(image, 1, (1, 2))
        # return torch.rot90(tmp, 1, (1, 2))

    def batch_augment(self, images):
        tmp = np.rot90(images, k=1, axes=(1, 2))
        tmp = np.rot90(tmp, k=1, axes=(1, 2))
        # tmp = np.rot90(images, k=1, axes=(2, 3))
        # tmp = np.rot90(tmp, k=1, axes=(2, 3))
        return tmp
        # tmp = torch.rot90(images, 1, (2, 3))
        # return torch.rot90(tmp, 1, (2, 3))

    def deaugment_boxes(self, boxes):
        tmp = TTARotate90().deaugment_boxes(boxes)
        return TTARotate90().deaugment_boxes(tmp)


class TTARotate270(BaseWheatTTA):
    def augment(self, image):
        tmp = TTARotate180().augment(image)
        tmp = np.rot90(tmp, k=1, axes=(0, 1))
        return tmp
        # return torch.rot90(tmp, 1, (1, 2))

    def batch_augment(self, images):
        tmp = TTARotate180().batch_augment(images)
        tmp = np.rot90(tmp, k=1, axes=(1, 2))
        return tmp
        # return torch.rot90(tmp, 1, (2, 3))

    def deaugment_boxes(self, boxes):
        tmp = TTARotate180().deaugment_boxes(boxes)
        return TTARotate90().deaugment_boxes(tmp)


class TTACompose(BaseWheatTTA):
    """ author: @shonenkov """
    def __init__(self, transforms):
        self.transforms = transforms

    def augment(self, image):
        for transform in self.transforms:
            image = transform.augment(image)
        return image

    def batch_augment(self, images):
        for transform in self.transforms:
            images = transform.batch_augment(images)
        return images

    def prepare_boxes(self, boxes):
        result_boxes = boxes.copy()
        result_boxes[:, 0] = np.min(boxes[:, [0, 2]], axis=1)
        result_boxes[:, 2] = np.max(boxes[:, [0, 2]], axis=1)
        result_boxes[:, 1] = np.min(boxes[:, [1, 3]], axis=1)
        result_boxes[:, 3] = np.max(boxes[:, [1, 3]], axis=1)
        return result_boxes

    def deaugment_boxes(self, boxes):
        for transform in self.transforms[::-1]:
            boxes = transform.deaugment_boxes(boxes)
        return self.prepare_boxes(boxes)

## Model Loading Test

In [ ]:
# import darknet as dn
# meta_file = b"/workspace/Github/darknet/build/darknet/x64/data/wheat_notebook.data"
# cfg = b"/workspace/Github/darknet/build/darknet/x64/yolov4-mish-416-wheat-v3.cfg"
# weights = b"/workspace/Github/darknet/build/darknet/x64/backup/yolov4-mish-416-wheat-v3_best.weights"
# net = dn.load_net_custom(cfg, weights, 0, 1)
# meta = dn.load_meta(meta_file)

In [ ]:
# net

In [ ]:
# dn.free_network_ptr(net)

## Predict Test Dataset

In [ ]:
def run_wbf(preds, image_size=image_width,
            iou_thr=0.55, skip_box_thr=0.7, weights=None):
    boxes = [(p['boxes'] / (image_size-1)).tolist() for p in preds]
    scores = [p['scores'].tolist() for p in preds]
    labels = [np.ones(p['scores'].shape[0]).astype(int).tolist() for p in preds]
    
    # print(boxes)
    boxes, scores, labels = ensemble_boxes_wbf.weighted_boxes_fusion(
        boxes, scores, labels, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [ ]:
from itertools import product

tta_transforms = []

for tta_combination in product([TTAHorizontalFlip(), None], 
                               [TTAVerticalFlip(), None],
                               [TTARotate90(), None]):
    tta_transforms.append(
        TTACompose([
            tta_transform for tta_transform in tta_combination if tta_transform
        ]))

In [ ]:
def detect(net, meta, darknet_image, target_image, prob_threshold=.5):
    dn.copy_image_from_bytes(darknet_image, target_image.tobytes())
    return dn.detect_image(net,
                           meta,
                           darknet_image,
                           thresh=prob_threshold,
                           hier_thresh=.5,
                           nms=.45)


# Modified from: https://www.kaggle.com/nvnnghia/yolov4-inference
def predict_test(batch_size=1, prob_threhold=0.5):
    image_names = os.listdir(test_image_folder)

    # Store results by image
    image_pred_results = collections.defaultdict(list)
    for model_index, cfg in enumerate(model_cfg_files):
        print(f"Generating inference for model cfg file {cfg} ......")

        weights = model_weights_files[model_index]
        net = dn.load_net_custom(cfg, weights, 0, batch_size)
        meta = dn.load_meta(meta_file)

        resize = model_image_sizes[model_index]

        # Create an image we reuse for each detect
        darknet_image = dn.make_image(resize, resize, 3)

        for name in image_names:
            image_id = name.split('.')[0]

            image = cv2.imread(f'{test_image_folder}/{image_id}.jpg')
            image = cv2.resize(image, (resize, resize))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            infer_start = time.time()

            # Test time augmentation
            for tta in tta_transforms:
                tta_image = tta.augment(image)

                preds = detect(net, meta, darknet_image, tta_image)
                scores, preds = process_detection_result(
                    preds, resize, prob_threhold)

                # Convert back to the orignal coordinations
                preds = tta.deaugment_boxes(preds)
                preds = preds.clip(min=0, max=image_width - 1)

                image_pred_results[name].append({
                    'boxes': preds,
                    'scores': scores,
                })

            print(
                f"Time spent on augmented inference for {name}: {time.time() - infer_start:2f} seconds"
            )

            del image, preds
            gc.collect()

        dn.free_network_ptr(net)
        del net, meta
        gc.collect()

    return image_pred_results


def ensemble(image_pred_results, iou_thr=0.5, skip_box_thr=0.1):
    ensemble_results = {}

    image_names = os.listdir(test_image_folder)
    for name in image_names:
        print(f"Running ensemble by WBF for {name} ......")
        image_id = name.split('.')[0]

        tta_preds = image_pred_results[name]

        boxes, scores, labels = run_wbf(tta_preds,
                                        iou_thr=iou_thr,
                                        skip_box_thr=skip_box_thr)
        boxes = boxes.round().astype(np.int32).clip(min=0, max=image_width - 1)

        # Convert to width and height
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

        ensemble_results[name] = {
            'boxes': boxes,
            'scores': scores,
        }

    return ensemble_results


def generate_submit(ensemble_results):
    results = []

    count = 0
    image_names = os.listdir(test_image_folder)
    for name in image_names:
        image_id = name.split('.')[0]

        if len(image_names) < 11:
            image = cv2.imread(f'{test_image_folder}/{image_id}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        r = ensemble_results[name]
        boxes, scores = r["boxes"], r["scores"]

        results.append({
            'image_id':
            image_id,
            'PredictionString':
            format_prediction_string(boxes, scores)
        })

        if len(image_names) < 11 and count < 10:
            fig, ax = plt.subplots(1, 1, figsize=(20, 10))
            for box, score in zip(boxes, scores):
                cv2.rectangle(image, (box[0], box[1], box[2], box[3]),
                              color=(99, 228, 255),
                              thickness=4)
                cv2.putText(
                    image,
                    f"{score:.2f}",
                    (box[0], box[1] + box[3] - 5),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8,  # fontScale
                    (255, 255, 255),
                    2,  # thickness
                    cv2.LINE_AA)
            ax.imshow(image)
            count += 1

    return results

In [ ]:
pred_start = time.time()
test_preds = predict_test(batch_size=1, prob_threhold=prob_threhold)
print(
    f"Time spent on generating submission predictions: {time.time() - pred_start:2f} seconds"
)

In [ ]:
pred_start = time.time()
ensemble_results = ensemble(test_preds,
                            iou_thr=iou_thr,
                            skip_box_thr=skip_box_thr)
print(f"Time spent on ensemble: {time.time() - pred_start:2f} seconds")

## Pseudo Labeling

In [ ]:
!mkdir -p data/wheat backup

### Create Dataset Files

In [ ]:
with open("./data/wheat.names", 'w') as out:
    out.write("wheat\n")

In [ ]:
!cat ./data/wheat.names

In [ ]:
with open(f"./data/wheat_pseudo.data", 'w') as out:
    out.write("classes= 1\n")
    out.write(f"train = data/train_wheat_pseudo.txt\n")
    out.write(f"valid = data/valid_wheat_pseudo.txt\n")
    out.write("names  = data/wheat.names\n")
    out.write("backup = backup")

In [ ]:
!cat ./data/wheat_pseudo.data

In [ ]:
fold_id = 0

fold_train_df, fold_val_df = df_folds[~(df_folds["fold"] == fold_id)].copy(
), df_folds[df_folds["fold"] == fold_id].copy()

assert fold_train_df.shape[0] + fold_val_df.shape[0] == df_folds.shape[0]
fold_train_df.shape[0], fold_val_df.shape[0]

In [ ]:
with open(f"./data/train_wheat_pseudo.txt", 'w') as out:
    training_image_ids = []
    for image_id in fold_train_df["image_id"].unique():
        training_image_ids.append((image_id, True))

    for name in os.listdir(test_image_folder):
        image_id = name.split('.')[0]
        training_image_ids.append((image_id, False))

    # Shuffle images to avoid overfitting
    random.shuffle(training_image_ids)

    for (image_id, is_train) in training_image_ids:
        if is_train:
            sh.copy(f"{train_image_folder}/{image_id}.jpg",
                    f"./data/wheat/{image_id}.jpg")
            out.write(f"./data/wheat/{image_id}.jpg\n")
        else:
            # Resize to ensure correct size
            image = cv2.imread(f"{test_image_folder}/{image_id}.jpg")
            image = cv2.resize(image, (image_width, image_height))
            cv2.imwrite(f"./data/wheat/{image_id}.jpg", image)
            out.write(f"./data/wheat/{image_id}.jpg\n")

In [ ]:
!ls -la ./data/wheat | head -n 10

In [ ]:
# !cat ./data/train_wheat_pseudo.txt

In [ ]:
with open(f"./data/valid_wheat_pseudo.txt", 'w') as out:
    image_ids_with_head = fold_val_df["image_id"].unique()
    for image_id in image_ids_with_head:
        sh.copy(f"{train_image_folder}/{image_id}.jpg",
                f"./data/wheat/{image_id}.jpg")
        out.write(f"./data/wheat/{image_id}.jpg\n")

In [ ]:
# !cat ./data/valid_wheat_pseudo.txt

In [ ]:
for image_id in df_folds["image_id"].unique():
    df = dataset_df[dataset_df["image_id"] == image_id].copy()
    with open(f"./data/wheat/{image_id}.txt", 'w') as out:
        for index, row in df.iterrows():
            if row['x_center'] > 0 and row[
                    'y_center'] and row['w'] > 0 and row['h'] > 0:

                out.write(f"{row['classes']} " +
                          f"{row['x_center']/image_width} " +
                          f"{row['y_center']/image_height} " +
                          f"{row['w']/image_width} " +
                          f"{row['h']/image_height}\n")

In [ ]:
len(ensemble_results)

In [ ]:
for name in os.listdir(test_image_folder):
    image_id = name.split('.')[0]

    with open(f"./data/wheat/{image_id}.txt", 'w') as out:
        r = ensemble_results[name]
        boxes, scores = r["boxes"], r["scores"]

        # Convert (x, y, w, h) to (center x, center y, w, h)
        boxes[:, 0] = boxes[:, 0] + boxes[:, 2] / 2
        boxes[:, 1] = boxes[:, 1] + boxes[:, 3] / 2

        print(boxes.shape)
        for i in range(boxes.shape[0]):
            box = boxes[i, :]
            if box[0] > 0 and box[1] > 0 and box[2] > 0 and box[3] > 0:
                out.write(f"0 " + f"{box[0]/image_width} " +
                          f"{box[1]/image_height} " +
                          f"{box[2]/image_width} " +
                          f"{box[3]/image_height}\n")

In [ ]:
# os.listdir(test_image_folder)

In [ ]:
# !cat ./data/wheat/2fd875eaa.txt

In [ ]:
# !cat ./data/wheat/51b3e36ab.txt | wc -l

In [ ]:
# ! cat ./data/wheat/{os.listdir(train_image_folder)[0][:-4]}.txt | wc -l

In [ ]:
# os.listdir(test_image_folder)

In [ ]:
# !cat ./data/wheat/51b3e36ab.txt | wc -l

### Create Config File

In [ ]:
cfg_content = f"""
# [V3]
# With pseudo labels
# mosaic=0

[net]
batch={batch}
subdivisions={subdivisions}

width=608
height=608

channels=3
momentum=0.949
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.0005
# learning_rate=0.001

burn_in={burn_in}

max_batches = {iteration_offset+pseudo_iterations}
policy=steps
steps={int(0.8*(iteration_offset+pseudo_iterations))},{int(0.9*(iteration_offset+pseudo_iterations))}
scales=.1,.1

# mosaic=1
blur=1
gaussian_noise=1
# https://github.com/AlexeyAB/darknet/issues/4446
# cutmix=1 # for training Classifier
# mixup=1 # for training Classifier

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1,-7

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1,-10

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=256
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1,-28

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=512
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1,-28

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

# Downsample

[convolutional]
batch_normalize=1
filters=1024
size=3
stride=2
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=mish

[shortcut]
from=-3
activation=linear

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1,-16

[convolutional]
batch_normalize=1
filters=1024
size=1
stride=1
pad=1
activation=mish

##########################

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

### SPP ###
[maxpool]
stride=1
size=5

[route]
layers=-2

[maxpool]
stride=1
size=9

[route]
layers=-4

[maxpool]
stride=1
size=13

[route]
layers=-1,-3,-5,-6
### End SPP ###

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[upsample]
stride=2

[route]
layers = 85

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1, -3

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[upsample]
# stride=4
stride=2

[route]
# layers = 23
layers = 54

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[route]
layers = -1, -3

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=256
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=256
activation=mish

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=mish

##########################

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=256
activation=mish

[convolutional]
size=1
stride=1
pad=1
filters=18
activation=linear

[yolo]
mask = 0,1,2
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
classes=1
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1
scale_x_y = 1.2
iou_thresh=0.213
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
nms_kind=greedynms
beta_nms=0.6

[route]
layers = -4

[convolutional]
batch_normalize=1
size=3
stride=2
# stride=4
pad=1
filters=256
activation=mish

[route]
layers = -1, -16

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=mish

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=512
activation=mish

[convolutional]
size=1
stride=1
pad=1
filters=18
activation=linear

[yolo]
mask = 3,4,5
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
classes=1
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1
scale_x_y = 1.1
iou_thresh=0.213
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
nms_kind=greedynms
beta_nms=0.6

[route]
layers = -4

[convolutional]
batch_normalize=1
size=3
stride=2
pad=1
filters=512
activation=mish

[route]
layers = -1, -37

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=mish

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=mish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1024
activation=mish

[convolutional]
size=1
stride=1
pad=1
filters=18
activation=linear

[yolo]
mask = 6,7,8
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
classes=1
num=9
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1
scale_x_y = 1.05
iou_thresh=0.213
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
nms_kind=greedynms
beta_nms=0.6

max=200
"""

In [ ]:
with open(f"./yolov4-mish-416-wheat-v3-pseudo.cfg", 'w') as out:
    out.write(f"{cfg_content}\n")

In [ ]:
# !cat ./yolov4-mish-416-wheat-v3-pseudo.cfg

### Start Pseudo Training

In [ ]:
# Redirect outputs to console
# import sys
# jupyter_console = sys.stdout
# sys.stdout = open('/dev/stdout', 'w')

# Append to log file
# sys.stdout = open(f"stdout.log", 'a')
# sys.stdout = jupyter_console

In [ ]:
# Local mode
# !./darknet detector train \
#     data/wheat_pseudo.data \
#     yolov4-mish-416-wheat-v3-pseudo.cfg \
#     {model_weights_files[0].decode('ascii')} \
#     -dont_show -mjpeg_port 8090 -map \
#     -gpus 1 \
#     -clear

In [ ]:
# Kernel mode
!./darknet detector train \
    data/wheat_pseudo.data \
    yolov4-mish-416-wheat-v3-pseudo.cfg \
    {model_weights_files[0].decode('ascii')} \
    -dont_show -mjpeg_port 8090 -map \
    -gpus 0 \
    -clear \
    > /dev/null 2>&1

### Generate New Test Predictions

In [ ]:
!ls -la ./backup

In [ ]:
# Modified from: https://www.kaggle.com/nvnnghia/yolov4-inference
def predict_new_test(batch_size=1, prob_threhold=0.5):
    image_names = os.listdir(test_image_folder)

    # Store results by image
    image_pred_results = collections.defaultdict(list)
    
    cfg = b"./yolov4-mish-416-wheat-v3-pseudo.cfg"
    # weights = b"./backup/yolov4-mish-416-wheat-v3-pseudo_last.weights"
    weights = b"./backup/yolov4-mish-416-wheat-v3-pseudo_best.weights"
    resize = 618
    
    print(f"Generating inference for model cfg file {cfg} ......")
    
    net = dn.load_net_custom(cfg, weights, 0, batch_size)
    meta = dn.load_meta(meta_file)

    # Create an image we reuse for each detect
    darknet_image = dn.make_image(resize, resize, 3)

    for name in image_names:
        image_id = name.split('.')[0]

        image = cv2.imread(f'{test_image_folder}/{image_id}.jpg')
        image = cv2.resize(image, (resize, resize))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        infer_start = time.time()

        # Test time augmentation
        for tta in tta_transforms:
            tta_image = tta.augment(image)

            preds = detect(net, meta, darknet_image, tta_image)
            scores, preds = process_detection_result(
                preds, resize, prob_threhold)

            # Convert back to the orignal coordinations
            preds = tta.deaugment_boxes(preds)
            preds = preds.clip(min=0, max=image_width - 1)

            image_pred_results[name].append({
                'boxes': preds,
                'scores': scores,
            })

        print(
            f"Time spent on augmented inference for {name}: {time.time() - infer_start:2f} seconds"
        )

        del image, preds
        gc.collect()

    dn.free_network_ptr(net)
    del net, meta
    gc.collect()

    return image_pred_results

In [ ]:
pred_start = time.time()
new_test_preds = predict_new_test(batch_size=1, prob_threhold=prob_threhold)
print(
    f"Time spent on generating submission predictions: {time.time() - pred_start:2f} seconds"
)

In [ ]:
pred_start = time.time()
new_ensemble_results = ensemble(new_test_preds,
                                iou_thr=iou_thr,
                                skip_box_thr=skip_box_thr)
print(f"Time spent on ensemble: {time.time() - pred_start:2f} seconds")

## Generate Submission

In [ ]:
pred_start = time.time()
final_results = generate_submit(new_ensemble_results)
final_results[:2]

In [ ]:
test_df = pd.DataFrame(final_results, columns=['image_id', 'PredictionString'])
test_df.head()
test_df.to_csv('submission.csv', index=False)

In [ ]:
test_df.head(10)

In [ ]:
! ls -la

In [ ]:
!rm -rf __pycache__ darknet* data libdarknet.so yolov4-* *.jpg

In [ ]:
! ls -la